In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from typing import TypedDict

In [2]:

llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-120b",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

/home/bumblebee/AgenticAI/venv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str

In [7]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [10]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [11]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()


In [12]:
intial_state = {'title': 'Rise of AI in India'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'title': 'Rise of AI in India', 'outline': '**Blog Title:** *The Rise of AI in India – Opportunities, Challenges, and What’s Next*  \n\n---\n\n## 1. Introduction  \n   - **Hook:** A striking statistic or anecdote (e.g., “India now hosts over 1,000 AI startups, and the AI‑driven market is projected to hit **$35\u202fbillion** by 2027”).  \n   - **Why it matters:** Link AI’s growth to India’s demographic dividend, economic ambitions, and global tech leadership.  \n   - **Purpose of the post:** Explain the forces driving AI adoption, showcase success stories, analyze challenges, and outline the road ahead.\n\n---\n\n## 2. Historical Context – From Early Experiments to a National Agenda  \n   ### 2.1 Early Foundations (1990s‑2000s)  \n   - Government research labs (C-DAC, ISRO) and academia (IITs, IISc) start AI research.  \n   - First Indian AI patents and publications.  \n   ### 2.2 Policy Milestones  \n   - **2009:** National Knowledge Network & Digital India launch – infrastructure gr

In [14]:
print(final_state['outline'])

**Blog Title:** *The Rise of AI in India – Opportunities, Challenges, and What’s Next*  

---

## 1. Introduction  
   - **Hook:** A striking statistic or anecdote (e.g., “India now hosts over 1,000 AI startups, and the AI‑driven market is projected to hit **$35 billion** by 2027”).  
   - **Why it matters:** Link AI’s growth to India’s demographic dividend, economic ambitions, and global tech leadership.  
   - **Purpose of the post:** Explain the forces driving AI adoption, showcase success stories, analyze challenges, and outline the road ahead.

---

## 2. Historical Context – From Early Experiments to a National Agenda  
   ### 2.1 Early Foundations (1990s‑2000s)  
   - Government research labs (C-DAC, ISRO) and academia (IITs, IISc) start AI research.  
   - First Indian AI patents and publications.  
   ### 2.2 Policy Milestones  
   - **2009:** National Knowledge Network & Digital India launch – infrastructure groundwork.  
   - **2018:** NITI Aayog’s *Artificial Intelligence S

In [15]:
print(final_state['content'])

# The Rise of AI in India – Opportunities, Challenges, and What’s Next  

*By [Your Name]* – *[Date]*  

---

## 1. Introduction  

**Hook:** *“India now hosts more than 1,200 AI‑focused startups, and the AI‑driven market is projected to reach **$35 billion** by 2027.”*  

That single figure captures a seismic shift that is already reshaping the country’s economy, its labor market, and its place on the global tech stage.  

### Why it matters  

- **Demographic dividend:** With a median age of just 28 years, India boasts a massive, English‑speaking talent pool that can train, build, and scale AI systems at a fraction of the cost of western counterparts.  
- **Economic ambition:** The government’s “Digital India” and “Make in India” missions depend on AI to boost productivity in traditionally labor‑intensive sectors such as agriculture, manufacturing, and logistics.  
- **Global tech leadership:** From Bengaluru’s “Silicon Valley of India” vibe to Hyderabad’s emerging “Cyberabad” ecosys